In [1]:
class CFG:
    exp = '022'
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + CFG.exp
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
CPU times: user 26.2 ms, sys: 29.7 ms, total: 55.9 ms
Wall time: 1.66 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [5]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{CFG.exp}')
output_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/stacking{CFG.exp}')
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [6]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
print(get_score(oof_df['score'], oof_df['pred']))
oof_df.head()

0.8316848277826169


,id,anchor,target,context,score,context_text,fold,section,sectok,text,pred
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]abatement of pollution.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.126895
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]act of abating.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.632196
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]active catalyst.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.237127
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]eliminating process.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.489984
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]forest region.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.013014


In [8]:
tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')

In [9]:
train = oof_df.copy()

# 特徴量エンジニアリング
def tok_len(sentence):
    return len(tokenizer.tokenize(sentence))

def feature_engineering(df):
    df['section'] = df['context'].map(lambda s: s[0])

    df['anchor_tok_len'] = df['anchor'].map(tok_len)
    df['target_tok_len'] = df['target'].map(tok_len)
    df['context_tok_len'] = df['context_text'].map(tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    return df


train = feature_engineering(train)
train.head()

,id,anchor,target,context,score,context_text,fold,section,sectok,text,pred,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]abatement of pollution.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.126895,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]act of abating.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.632196,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]active catalyst.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.237127,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]eliminating process.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.489984,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]forest region.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.013014,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10


In [10]:
# カテゴリ変数の変換
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [11]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'context_text', 'fold', 'section', 'sectok', 'text', 'pred', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded'], dtype='object')


In [12]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'section', 'sectok']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['pred', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded']


In [13]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [14]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [15]:
best_params = tuning_params(train, params)
best_params

Output hidden; open in https://colab.research.google.com to view.

In [16]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [17]:
pred = train_fn(train, best_params)

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0308076	valid_1's l2: 0.0297138
[200]	training's l2: 0.022428	valid_1's l2: 0.0210508
[300]	training's l2: 0.0209415	valid_1's l2: 0.0194776
[400]	training's l2: 0.0205728	valid_1's l2: 0.0191425
[500]	training's l2: 0.0204392	valid_1's l2: 0.0190669
[600]	training's l2: 0.0203633	valid_1's l2: 0.0190565
Early stopping, best iteration is:
[576]	training's l2: 0.0203766	valid_1's l2: 0.0190517
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0302775	valid_1's l2: 0.0304539
[200]	training's l2: 0.0217979	valid_1's l2: 0.0223743
[300]	training's l2: 0.0202868	valid_1's l2: 0.0211439
[400]	training's l2: 0.0199206	valid_1's l2: 0.0209541
[500]	training's l2: 0.0197983	valid_1's l2: 0.0209445
Early stopping, best iteration is:
[496]	training's l2: 0.0198018	valid_1's l2: 0.0209441
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0305517

In [18]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

,feature,importance
0,pred,49689.577319
2,target_tok_len,632.271364
11,section_encoded,292.569461
6,len_anc_tgt_div,180.191539
1,anchor_tok_len,163.817710
5,len_anc_tgt_diff,142.564898
10,len_tgt_cnt_div,41.828953
8,len_anc_cnt_div,29.416130
9,len_tgt_cnt_diff,23.874721
4,input_len,18.924455


In [19]:
# optunaのbest params
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8316848277826169
0.8334362295451807


,id,anchor,target,context,score,context_text,fold,section,sectok,text,pred,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]abatement of pollution.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.126895,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0,0.203237
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]act of abating.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.632196,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0,0.599836
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]active catalyst.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.237127,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.264313
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]eliminating process.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.489984,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.478604
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,A,[A],abatement.[s]forest region.[s]HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0.013014,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0,0.111066


In [20]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()

Starting upload for file section_encoder.pkl
100% 323/323 [00:02<00:00, 130B/s]
Upload successful: section_encoder.pkl (323B)
Starting upload for file cols.pkl
100% 256/256 [00:02<00:00, 85.8B/s]
Upload successful: cols.pkl (256B)
Starting upload for file lgb_fold0.txt
100% 455k/455k [00:03<00:00, 146kB/s]
Upload successful: lgb_fold0.txt (455KB)
Starting upload for file lgb_fold1.txt
100% 392k/392k [00:02<00:00, 156kB/s]
Upload successful: lgb_fold1.txt (392KB)
Starting upload for file lgb_fold2.txt
100% 557k/557k [00:03<00:00, 151kB/s]
Upload successful: lgb_fold2.txt (557KB)
Starting upload for file lgb_fold3.txt
100% 460k/460k [00:02<00:00, 176kB/s]
Upload successful: lgb_fold3.txt (460KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/hanejiyuto/PPPM-stacking-exp022
